In [1]:
username = "tester"
password  = "password"
url = "bolt://localhost:7687"
database="ctgov"
node_label= "AdverseEvent", 
embedding_node_property="biobert_emb"
index_name = "adverse_event"
text_node_properties = ["term","organ_system"]
user_query = "Anaemia"

In [9]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding as LI_HF

from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex

llm = Ollama(model="mistral", request_timeout=30.0)
li_biobert = LI_HF(model_name="dmis-lab/biobert-base-cased-v1.1")

Settings.llm = llm
Settings.embed_model = li_biobert

neo4j_vector = Neo4jVectorStore(
    username,
    password,
    url,
    768,
    node_label= node_label, 
    embedding_node_property= embedding_node_property, 
    text_node_property= text_node_properties[0],
    index_name=index_name,
    keyword_index_name= "keyword",
    hybrid_search=True
)

loaded_index = VectorStoreIndex.from_vector_store(neo4j_vector).as_query_engine()
test= loaded_index.query(user_query)
print(test)

Empty Response


In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import HuggingFaceEmbeddings as LC_HF

lc_biobert = LC_HF(model_name="dmis-lab/biobert-base-cased-v1.1")

adverse_event = Neo4jVector.from_existing_graph(
    username=username, 
    password=password, 
    database=database,
    url=url,
    node_label= node_label, 
    embedding_node_property= embedding_node_property, 
    text_node_properties= text_node_properties,
    index_name=index_name,
    search_type="hybrid",
    keyword_index_name= "adverse_event_kw",
    embedding= lc_biobert,
    )

test = adverse_event.similarity_search_with_score(user_query, k=3)
print(test)

text2sql